#### Data Science Capstone Project,  Springboard Bootcamp <br> Title: "Improving Restaurant Reputation Using Yelp User Reviews" <br> Reza Taeb <br> San Francisco, Spring 2018 

## Part 4 - Machine Learning

In [1]:
# Importing Necessary Packages and Libraries

import pandas as pd 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Adjust output view   # May be it can be deleted 

pd.set_option('display.width', 115)
pd.options.display.max_colwidth = 30
sns.set()

First, Let's load the restaurant and review datasets and review datasets that have been modified in the previous parts.    

*** (" restaurant.csv & restaurant_eng.csv & review_restaurant_eng.csv ") ***

In [3]:
# read cleaned CSV files ("review_restaurant_eng", "restaurant_eng") 

df_review_restaurant_eng_processed = pd.read_csv('../review_restaurant_eng_processed.csv')
df_review_restaurant_eng_small_processed = pd.read_csv('../review_restaurant_eng_small_processed.csv')
df_restaurant_eng = pd.read_csv('../restaurant_eng.csv')

In [4]:
# General Information of the two datasets : 

print (df_restaurant_eng.info())
print (df_review_restaurant_eng_processed.info())
print (df_review_restaurant_eng_small_processed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48564 entries, 0 to 48563
Data columns (total 17 columns):
business_id     48564 non-null object
name            48564 non-null object
neighborhood    22075 non-null object
address         48266 non-null object
city            48564 non-null object
state           48564 non-null object
postal_code     48470 non-null object
latitude        48564 non-null float64
longitude       48564 non-null float64
stars           48564 non-null float64
review_count    48564 non-null int64
is_open         48564 non-null int64
attributes      48564 non-null object
categories      48564 non-null object
hours           48564 non-null object
country         48564 non-null object
food_type       23281 non-null object
dtypes: float64(3), int64(2), object(12)
memory usage: 6.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2957735 entries, 0 to 2957734
Data columns (total 11 columns):
review_id      object
user_id        object
business_id    obje

In [5]:
# 

print (df_review_restaurant_eng_small_processed.head(5))


                review_id                 user_id             business_id  stars        date  \
0  oZf4WJtIdGFbz62mW1qFWA  ngOgZ135Yp2FjRJ_KYgCtg  nDZWoEznY5-oSCc2HvoMtA      1  2014-02-24   
1  qj3M6xZeAXDhLXkZv7yVnA  bEKLZqj3gR5zyQDVlquaDg  YRyYbOSwvHkZsZOLv98oQg      1  2012-02-05   
2  CdGmiAx-3u-P93PxpqjpSQ  Q8CLfv38ArMpPvv-8Wi-gQ  Qa4eXuZ1IFPwnVXJcpZWtw      4  2013-12-12   
3  JpHPWaATFPG8Med9y5aZHw  ZHIMwodx9MIrWbJ8AyUbcg  iUYAqWKoBdHqYNMDzOXb9w      5  2015-04-26   
4  Cw5Py-cW1lRB6seZLlWVxQ  zNx6JRMhUHoIhLdKMjN4SA  zaAh8JjNeDyrJXf6xNkdUQ      5  2016-07-15   

                            text  useful  funny  cool                      word_list  word_count  \
0  really disappointed in thi...     0.0    0.0   0.0  ['really', 'disappointed',...         193   
1  service was terrible had t...     0.0    0.0   0.0  ['service', 'was', 'terrib...          74   
2  you probably already know ...     1.0    4.0   0.0  ['you', 'probably', 'alrea...         486   
3  simply unbelievable 

## 3 & 4 stars

From here, I am trying to figure out whether we can distinguish the 3 stars and 4 stars reviews by just checking the “text” of reviews or not. Therfore, I am going to focus just on the 3 and 4 stars reviews. 

In [6]:
# Filtering the 3 and 4 star reviews: 

three_four_star_restaurants = df_review_restaurant_eng_small_processed[df_review_restaurant_eng_small_processed['stars'].isin(['4','3'])]

In [7]:
# Check the 3 & 4 star ratings : 

three_four_star_restaurants.info()
print (three_four_star_restaurants.head(5))
print (three_four_star_restaurants['stars'].value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6112 entries, 2 to 14788
Data columns (total 14 columns):
review_id               6112 non-null object
user_id                 6112 non-null object
business_id             6112 non-null object
stars                   6112 non-null int64
date                    6112 non-null object
text                    6112 non-null object
useful                  6112 non-null float64
funny                   6112 non-null float64
cool                    6112 non-null float64
word_list               6112 non-null object
word_count              6112 non-null int64
text_modified           6112 non-null object
Ngrams_list             6112 non-null object
ngrams_list_modified    6112 non-null object
dtypes: float64(3), int64(2), object(9)
memory usage: 716.2+ KB
                 review_id                 user_id             business_id  stars        date  \
2   CdGmiAx-3u-P93PxpqjpSQ  Q8CLfv38ArMpPvv-8Wi-gQ  Qa4eXuZ1IFPwnVXJcpZWtw      4  2013-12-12   
11 

Since there are almost double 4 star entires than 3 star entries, it's better to make the size of them equal (** Under Sampling **) before going through ML algorithm :

In [8]:
# Make an equal sample of 3 and 4 star entries 

no_three_star = len(three_four_star_restaurants[three_four_star_restaurants['stars']==3])
four_star_indices =  three_four_star_restaurants[three_four_star_restaurants.stars == 4].index

# Random sample of "4 star" ratings 

random_indices = np.random.choice(four_star_indices, no_three_star , replace=False)
three_star_indices = three_four_star_restaurants[three_four_star_restaurants.stars == 3].index

# Concat 3 stars indices with 4 star ones

under_sample_indices = np.concatenate([three_star_indices,random_indices])

# Get Balance Dataframe

three_four_star_restaurants_balanced = three_four_star_restaurants.loc[under_sample_indices]

# check it out 

print (three_four_star_restaurants_balanced['stars'].value_counts())

3    2125
4    2125
Name: stars, dtype: int64


In [9]:
# Define text processing function 

def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [10]:
# Define X and y for further steps (Dependent adn Independent variables)

X = three_four_star_restaurants_balanced['word_list']
y = three_four_star_restaurants_balanced['stars']

In [11]:
# Vectorisation 

bow_transformer = CountVectorizer().fit(X)
X = bow_transformer.transform(X)

In [12]:
# Split the dataset into training and test sets 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

** Multinomial Naive Bayes ** is a specialised version of Naive Bayes designed more for text documents. Let’s build a Multinomial Naive Bayes model and fit it to our training set (X_train and y_train).

In [13]:
# Training our model

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Our model has now been trained! It’s time to see how well it predicts the ratings of previously unseen reviews (reviews from the test set). First, let’s store the predictions as a separate dataframe called ** predicts. **

In [14]:
# Testing and evaluating our model

predicts = nb.predict(X_test)

Next, let’s evaluate our predictions against the actual ratings (stored in y_test) using confusion_matrix and classification_report from Scikit-learn.

In [15]:
# Evaluate the predictions against actual ratings 

from sklearn.metrics import confusion_matrix, classification_report
print ('Confusion Matrix')
print(confusion_matrix(y_test, predicts))
print('\n \n')
print ('        Classification Report (3 and 4 stars reviews)')
print('\n ')
print(classification_report(y_test, predicts))

Confusion Matrix
[[479 155]
 [198 443]]

 

        Classification Report (3 and 4 stars reviews)

 
             precision    recall  f1-score   support

          3       0.71      0.76      0.73       634
          4       0.74      0.69      0.72       641

avg / total       0.72      0.72      0.72      1275



## 4 & 5 stars

In [16]:
# Filtering the 4 and 5 star reviews: 

four_five_star_restaurants = df_review_restaurant_eng_small_processed[df_review_restaurant_eng_small_processed['stars'].isin(['4','5'])]

In [17]:
# Check the 4 & 5 star ratings : 

print (four_five_star_restaurants['stars'].value_counts())

5    5546
4    3987
Name: stars, dtype: int64


Balancing ( ** undersampling ** ) the 4 and 5 star ratings entries: 

In [18]:
# Make an equal sample of 4 and 5 star entries 

no_four_star = len(four_five_star_restaurants[four_five_star_restaurants['stars']==4])
five_star_indices =  four_five_star_restaurants[four_five_star_restaurants.stars == 5].index

# Random sample of "5 star" ratings 

random_indices = np.random.choice(five_star_indices, no_four_star , replace=False)
four_star_indices = four_five_star_restaurants[four_five_star_restaurants.stars == 4].index

# Concat 3 stars indices with 4 star ones

under_sample_indices = np.concatenate([four_star_indices,random_indices])

# Get Balance Dataframe

four_five_star_restaurants_balanced = four_five_star_restaurants.loc[under_sample_indices]

# check it out 

print (four_five_star_restaurants_balanced['stars'].value_counts())

5    3987
4    3987
Name: stars, dtype: int64


In [19]:
# Define X and y for further steps 

X = four_five_star_restaurants_balanced['word_list']
y = four_five_star_restaurants_balanced['stars']

In [20]:
# Vectorisation 

bow_transformer = CountVectorizer().fit(X)
X = bow_transformer.transform(X)

In [21]:
# Split the dataset into training and test sets 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [22]:
# Training our model

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
# Testing and evaluating our model

predicts = nb.predict(X_test)

In [24]:
# Evaluate the predictions against actual ratings 

from sklearn.metrics import confusion_matrix, classification_report
print ('Confusion Matrix')
print(confusion_matrix(y_test, predicts))
print('\n \n')
print ('        Classification Report (4 and 5 stars reviews)')
print('\n ')
print(classification_report(y_test, predicts))

Confusion Matrix
[[813 378]
 [332 870]]

 

        Classification Report (4 and 5 stars reviews)

 
             precision    recall  f1-score   support

          4       0.71      0.68      0.70      1191
          5       0.70      0.72      0.71      1202

avg / total       0.70      0.70      0.70      2393



## 3 & 5 stars

In [25]:
# Filtering the 3 and 5 star reviews: 

three_five_star_restaurants = df_review_restaurant_eng_small_processed[df_review_restaurant_eng_small_processed['stars'].isin(['3','5'])]

In [26]:
# Check the 3 & 5 star ratings : 

print (three_five_star_restaurants['stars'].value_counts())

5    5546
3    2125
Name: stars, dtype: int64


Balancing ( ** undersampling ** ) the 3 and 5 star ratings entries: 

In [27]:
# Make an equal sample of 3 and 5 star entries 

no_three_star = len(three_five_star_restaurants[three_five_star_restaurants['stars']==3])
five_star_indices =  three_five_star_restaurants[three_five_star_restaurants.stars == 5].index

# Random sample of "5 star" ratings 

random_indices = np.random.choice(five_star_indices, no_three_star , replace=False)
three_star_indices = three_five_star_restaurants[three_five_star_restaurants.stars == 3].index

# Concat 3 stars indices with 4 star ones

under_sample_indices = np.concatenate([three_star_indices,random_indices])

# Get Balance Dataframe

three_five_star_restaurants_balanced = three_five_star_restaurants.loc[under_sample_indices]

# check it out 

print (three_five_star_restaurants_balanced['stars'].value_counts())

3    2125
5    2125
Name: stars, dtype: int64


In [28]:
# Define X and y for further steps 

X = three_five_star_restaurants_balanced['word_list']
y = three_five_star_restaurants_balanced['stars']

In [29]:
# Vectorisation 

bow_transformer = CountVectorizer().fit(X)
X = bow_transformer.transform(X)

In [30]:
# Split the dataset into training and test sets 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [31]:
# Training our model

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
# Testing and evaluating our model

predicts = nb.predict(X_test)

In [33]:
# Evaluate the predictions against actual ratings 

from sklearn.metrics import confusion_matrix, classification_report
print ('Confusion Matrix')
print(confusion_matrix(y_test, predicts))
print('\n \n')
print ('        Classification Report (3 and 5 stars reviews)')
print('\n ')
print(classification_report(y_test, predicts))

Confusion Matrix
[[569  65]
 [123 518]]

 

        Classification Report (3 and 5 stars reviews)

 
             precision    recall  f1-score   support

          3       0.82      0.90      0.86       634
          5       0.89      0.81      0.85       641

avg / total       0.86      0.85      0.85      1275

